This example uses SpyDrNet TMR Open-Source tool to generate Triple Modular Redundant EDIFs.

In [ ]:
pip install spydrnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 461 kB 5.1 MB/s 


In [ ]:
!git clone https://github.com/byuccl/spydrnet-tmr.git
%cd /content/spydrnet-tmr

Cloning into 'spydrnet-tmr'...
remote: Enumerating objects: 1674, done.
remote: Counting objects: 100% (317/317), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 1674 (delta 139), reused 224 (delta 107), pack-reused 1357
Receiving objects: 100% (1674/1674), 7.21 MiB | 18.05 MiB/s, done.
Resolving deltas: 100% (1012/1012), done.
/content/spydrnet-tmr


In [ ]:
pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/spydrnet-tmr
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for spydrnet-tmr: filename=spydrnet_tmr-1.2.0-py3-none-any.whl size=62472 sha256=123188aed1a72229ce32552dbde4f738509305fa90bc5e6f4f74b1e5abc3ebab
  Stored in directory: /root/.cache/pip/wheels/9a/8b/89/ce2010072ad5befcc6f4f4efe9959f0c9ce12538644ba95d0b
Successfully built spydrnet-tmr


In [ ]:
!cp -a /content/spydrnet-tmr/spydrnet_tmr/support_files /usr/local/lib/python3.7/dist-packages/spydrnet_tmr/

In [ ]:
%cd /content/spydrnet-tmr/examples/basic/

/content/spydrnet-tmr/examples/basic


In [ ]:
! python3 plot_generic_tmr.py

Identified 0 insertion points for reduction voters.


In [ ]:
"""
Basic TMR
----------

A bare minimum use case for SpyDrNet-TMR.

This script loads in a netlist of an inverter from SpyDrNet. Only required parameters are given to `apply_tmr_to_netlist` so default settings will be applied. Everything in the netlist will be replicated, including all top-level ports. Since there are no reduction voters required and no special voter insertion was enabled, there are no voters inserted in this design.

"""

import spydrnet as sdn
from spydrnet_tmr.apply_tmr_to_netlist import apply_tmr_to_netlist
from spydrnet_tmr.support_files.vendor_names import XILINX


netlist_name = "inverter"
netlist = sdn.load_example_netlist_by_name(netlist_name)
apply_tmr_to_netlist(
    netlist,
    XILINX,
)
sdn.compose(netlist, netlist_name + "_tmr.edf")

Identified 0 insertion points for reduction voters.


In [ ]:
! python3 plot_xilinx_generic_tmr.py

Identified 4 insertion points for reduction voters.

Primitive count before and after TMR:
	 BUFG :  1  -->  3
	 IBUF :  4  -->  12
	 LUT1 :  1  -->  3
	 LUT3 :  1  -->  7
	 LUT4 :  1  -->  3
	 LUT5 :  1  -->  3
	 OBUF :  4  -->  12
	 FDCE :  4  -->  12
	 INV :  0  -->  0


In [ ]:
"""
Xilinx TMR
===========
This is a xilinx TMR example using SpyDrNet TMR

The number of times each primitive is instanced before and after triplicating will be printed.
"""

import spydrnet as sdn
from spydrnet.uniquify import uniquify
from spydrnet_tmr import apply_nmr, insert_organs
from spydrnet_tmr.support_files.vendor_names import XILINX
from spydrnet_tmr.apply_tmr_to_netlist import apply_tmr_to_netlist

# set_property design_mode GateLvl [current_fileset]
# set_property edif_top_file <path_to_file> [current_fileset]
# link_design -part <part_number> -mode out_of_context
netlist = sdn.load_example_netlist_by_name(
    "fourBitCounter"
)  # loading an example, use `sdn.parse(<netlist filename>)` otherwise

# uniquify is called to insure that non-leaf definitions are instanced only once, prevents unintended transformations.
uniquify(netlist)

# set instances_to_replicate [get_cells -hierarchical -filter {PRIMITIVE_LEVEL==LEAF||PRIMITIVE_LEVEL==MACRO}]
hinstances_to_replicate = list(
    netlist.get_hinstances(
        recursive=True, filter=lambda x: x.item.reference.is_leaf() is True
    )
)

# set ports_to_replicate [get_ports]
hports_to_replicate = list(
    netlist.get_hports(filter=lambda x: x.item.direction is sdn.IN)
)

valid_voter_point_dict = dict()
valid_voter_point_dict["reduction"] = [
    *netlist.get_hports(),
    *hinstances_to_replicate,
]

# find out where to insert reduction voters
netlist = apply_tmr_to_netlist(
    netlist,
    XILINX,
    hinstances_and_hports_to_replicate=[
        *hports_to_replicate,
        *hinstances_to_replicate,
    ],
    valid_voter_point_dict=valid_voter_point_dict,
)

# print the number of times each primitive is instanced
def instance_count(original_netlist, new_netlist):
    print("\nPrimitive count before and after TMR:")
    primitives_library = next(
        new_netlist.get_libraries("hdi_primitives"), None
    )
    for primitive in primitives_library.get_definitions():
        i = 0
        j = 0
        for instance in original_netlist.get_instances():
            if primitive.name == instance.reference.name:
                i += 1
        for instance in new_netlist.get_instances():
            if primitive.name == instance.reference.name:
                j += 1
        print("\t", primitive.name, ": ", i, " --> ", j)


instance_count(sdn.load_example_netlist_by_name("fourBitCounter"), netlist)

sdn.compose(netlist, "fourBitCounter_tmr.edf")

Identified 4 insertion points for reduction voters.

Primitive count before and after TMR:
	 BUFG :  1  -->  3
	 IBUF :  4  -->  12
	 LUT1 :  1  -->  3
	 LUT3 :  1  -->  7
	 LUT4 :  1  -->  3
	 LUT5 :  1  -->  3
	 OBUF :  4  -->  12
	 FDCE :  4  -->  12
	 INV :  0  -->  0


TODO: add real design example from Linux on Litex